In [ ]:
# Unzip the uploaded dataset
!unzip -o university_chatbot_dataset.zip


In [ ]:
with open("intents.json") as file:
    data = json.load(file)

# Display a sample
for intent in data["intents"]:
    print(intent)
    break


In [ ]:
!pip install nltk

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional: improves lemmatization
nltk.download('averaged_perceptron_tagger')  # Sometimes helpful
nltk.download('punkt_tab')


In [ ]:
import json
import pickle
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Load intents.json
with open('intents.json') as file:
    data = json.load(file)

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Fix for your dataset (text and intent instead of patterns and tag)
for intent in data['intents']:
    for pattern in intent['text']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['intent']))
        if intent['intent'] not in classes:
            classes.append(intent['intent'])

# Lemmatize and sort
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Save words and classes
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Create training data
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in document[0]]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])

# Shuffle and convert to numpy array
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Build model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile and train
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save model
model.save('model.h5')
print("✅ Model trained and saved successfully!")



In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Load all files
model = load_model('model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Clean up user input
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Convert input into bag of words
def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

# Predict the intent
def predict_class(sentence):
    bow_vector = bow(sentence, words)
    res = model.predict(np.array([bow_vector]))[0]
    threshold = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > threshold]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Get response from intents.json
def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        return "I'm not sure how to respond to that."
    tag = intents_list[0]['intent']
    for i in intents_json['intents']:
        if i['intent'] == tag:
            return random.choice(i['responses'])

# Chat with the bot
def chatbot_response(text):
    intents_list = predict_class(text)
    response = get_response(intents_list, intents)
    return response


In [ ]:
print(chatbot_response("hello"))
print(chatbot_response("can you help me?"))


In [ ]:
print("\nYou: can I get info about admission?")
print("Bot:", chatbot_response("can I get info about admission?"))

In [ ]:
print("\nYou: where  i can get the admit card")
print("Bot:", chatbot_response("where  i can get the admit card"))

In [ ]:
print("\nYou: what is the process of admission?")
print("Bot:", chatbot_response("what is the process of admission?"))

In [ ]:
import json

# Load intents.json
with open("intents.json", "r") as file:
    data = json.load(file)

# Search for intents related to "apply" or "admission"
keywords = ["apply", "admission", "procedure", "how to apply"]
found = False

print("🔍 Searching for relevant intents...\n")
for intent in data["intents"]:
    all_texts = intent.get("text", [])
    for phrase in all_texts:
        if any(keyword.lower() in phrase.lower() for keyword in keywords):
            print(f"✅ Found in intent: {intent['intent']}")
            print("Examples:", all_texts)
            print("Responses:", intent['responses'])
            found = True
            print("-" * 60)
            break

if not found:
    print("❌ No intent found for admission/apply-related questions.")


In [ ]:
print(chatbot_response("how do I apply?"))
print(chatbot_response("hello"))
print(chatbot_response("where is the campus?"))


In [ ]:
!pip freeze | grep -E 'pandas|tensorflow|matplotlib|seaborn|scikit-learn'
